In [1]:
#parameters

imgSizeA = 28
imgSizeB = 28

numTestSamples = 100
numResultImagesPacks = 100
numResultImagesInPack = 100
batchSize = 64
numOfLabels = 98

numEpochs = 1700

continueWorking = True 
# continueWorking = False
dataDir = 'data6000_BigNetwork_labels_gen_droput0_disc1layer_dropout0_adamBeta1_05_batchSize64'

In [3]:
#imports

%matplotlib inline
from helper import Helper
import torch
import torch.autograd.variable as Variable
from glob import glob
import os
from torchvision import transforms, datasets
from IPython import display
import numpy as np

In [ ]:
#continuation

#find newest model
if continueWorking:
    list_of_files = glob('./{}/savedModels/*'.format(dataDir)) # * means all if need specific format then *.csv
    latest_file = max(list_of_files, key=os.path.getctime).split("/")[-1]

    print(int(latest_file.split('_')[2]))
    epoch = int(latest_file.split('_')[2])

In [ ]:
#helpers

def generatorOutputToImages(vectors):
    return vectors.view(vectors.size(0), 1, imgSizeB, imgSizeA)

def imagesToDiscriminatorInput(images):
    return images.view(images.size(0), imgSizeA * imgSizeB)

def noise(numSamples):
    n = Variable(torch.randn(numSamples, 100))
    if torch.cuda.is_available(): return n.cuda() 
    return n

def ones(size):
    if torch.cuda.is_available(): return Variable(torch.ones(size, 1)).cuda()
    return Variable(torch.ones(size, 1))

def zeros(size):
    if torch.cuda.is_available(): return Variable(torch.zeros(size, 1)).cuda()
    return Variable(torch.zeros(size, 1))

def getLabelsForGenerator(size):
    return Variable(torch.LongTensor(np.random.randint(0, numOfLabels, size)))

In [ ]:
#Unpack data files. Needed when you run this program for the first time.
import shutil
import tarfile

def concatenateSplittedArch(pathToRawDir):
    with open(os.path.join(pathToRawDir, 'rawData.joined.tar.gz'),'wb') as wfd:
        for f in [os.path.join(pathToRawDir, 'rawData.tar.gz.partsaa'),os.path.join(pathToRawDir, 'rawData.tar.gz.partsab'),os.path.join(pathToRawDir, 'rawData.tar.gz.partsac')]:
            with open(f,'rb') as fd:
                shutil.copyfileobj(fd, wfd)
                
def unpackArch(pathToRawDir):
    tarFile = tarfile.open(os.path.join(pathToRawDir, 'rawData.joined.tar.gz'), "r:gz")
    tarFile.extractall(pathToRawDir)
    tarFile.close()
    
pathToRawDir = './generatedMNIST/MNIST/dataset/raw'
if not("t10k-images-idx3-ubyte" in os.listdir(pathToRawDir) and "t10k-labels-idx1-ubyte" in os.listdir(pathToRawDir) and "train-images-idx3-ubyte" in os.listdir(pathToRawDir) and "train-labels-idx1-ubyte" in os.listdir(pathToRawDir)):
    print("one or more MNIST imitating data file not found. Normal if run for the first time. Unpacking arch with data...")
    concatenateSplittedArch(pathToRawDir)
    unpackArch(pathToRawDir)
    print("done")


In [ ]:
#Dataset

# from optparse import OptionParser
# parser = OptionParser()
# parser.add_option("-f", "--mnistDataDir", dest="mnistDataDir", default="./generatedMNIST",
#                   help="path to mnist data without dataset/raw")

# (options, args) = parser.parse_args()
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

mnistData = datasets.MNIST(root='./generatedMNIST', train=True, transform=transform, download=False, process=True)

dataLoader = torch.utils.data.DataLoader(mnistData, batch_size=batchSize, shuffle=True)
numBatchesPerEpoch = len(dataLoader)


In [ ]:
#Neural Networks

class DiscriminatorNN(torch.nn.Module):
    def __init__(self):
        super(DiscriminatorNN, self).__init__()
        
        self.label_emb = torch.nn.Embedding(numOfLabels,numOfLabels)
        
        self.inSize = imgSizeA * imgSizeB + numOfLabels
        self.hidden0Size = 1024
        self.hidden1Size = 512
        self.hidden2Size = 256
        self.outSize = 1
        
        def hiddenLayerSequential(numIn, numOut):
            return torch.nn.Sequential (
            torch.nn.Linear(numIn, numOut),
            torch.nn.LeakyReLU(0.3),#TODO SZOZDA more leakyReLu?
            torch.nn.Dropout(0.2)#TODO SZOZDA dropout not in first layer maybe?
            )
        def hiddenLayerSequentialWithoutDropout(numIn, numOut):
            return torch.nn.Sequential (
            torch.nn.Linear(numIn, numOut),
            torch.nn.LeakyReLU(0.3),#TODO SZOZDA more leakyReLu?
            )
            
        self.layers = torch.nn.Sequential(
            hiddenLayerSequentialWithoutDropout(self.inSize, self.hidden0Size),
            hiddenLayerSequential(self.hidden0Size, self.hidden1Size),
            hiddenLayerSequential(self.hidden1Size, self.hidden2Size),
            torch.nn.Sequential(torch.nn.Linear(self.hidden2Size, self.outSize), torch.nn.Sigmoid())
        )
        
    def forward(self, x, labels):
        x = x.view(x.size(0), 784)
        c = self.label_emb(labels)
        x = torch.cat([x, c], 1)
        out = self.layers(x)
        return out.squeeze()

    
dNetwork = DiscriminatorNN()
if torch.cuda.is_available():
    dNetwork.cuda()

class GeneratorNN(torch.nn.Module):
    def __init__(self):
        super(GeneratorNN, self).__init__()
        self.label_emb = torch.nn.Embedding(numOfLabels,numOfLabels)
        
        self.inSize = 100 + numOfLabels
        self.hidden0Size = 256
        self.hidden1Size = 512
        self.hidden2Size = 1024
        self.outSize = imgSizeA * imgSizeB
        
        def hiddenLayerSequential(numIn, numOut):
            return torch.nn.Sequential(            
            torch.nn.Linear(numIn, numOut),
            torch.nn.LeakyReLU(0.3),
            # torch.nn.Dropout(0.5)
        )
        
        self.layers= torch.nn.Sequential(
            hiddenLayerSequential(self.inSize, self.hidden0Size),
            hiddenLayerSequential(self.hidden0Size, self.hidden1Size),
            hiddenLayerSequential(self.hidden1Size, self.hidden2Size),
            torch.nn.Sequential(torch.nn.Linear(self.hidden2Size, self.outSize), torch.nn.Tanh())
        )
        
    def forward(self, x, labels):
        x = x.view(x.size(0), 100)
        c = self.label_emb(labels)
        x = torch.cat([x, c], 1)
        return self.layers(x)

gNetwork = GeneratorNN()
if torch.cuda.is_available():
    gNetwork.cuda()

In [ ]:
#training methods

bceLoss = torch.nn.BCELoss()

discrOptimizer = torch.optim.Adam(dNetwork.parameters(), lr=0.0002, betas=(0.5, 0.999))
genOptimizer = torch.optim.Adam(gNetwork.parameters(), lr=0.0002, betas=(0.5, 0.999))

def calcLossAndBackward(samples, target, labelsFromBatch):
    prediction = dNetwork(samples, labelsFromBatch)
    error = bceLoss(prediction, target)
    error.backward()
    return prediction, error

def discriminatorTrain(optimizer, realSamples, generatedSamples, labelsFromBatch, labelsForGenerator):
    optimizer.zero_grad()
    
    predictionReal, errorReal = calcLossAndBackward(realSamples, ones(realSamples.size(0)), labelsFromBatch)
    predictionGen, errorGen= calcLossAndBackward(generatedSamples, zeros(generatedSamples.size(0)), labelsForGenerator)
    
    optimizer.step()
    
    return errorReal + errorGen, predictionReal, predictionGen

def generatorTrain(optimizer, generatedSamples, labelsForGenerator):
    optimizer.zero_grad()
    
    predictionGen, errorGen = calcLossAndBackward(generatedSamples, ones(generatedSamples.size(0)), labelsForGenerator)
    
    optimizer.step()
    
    return errorGen


In [ ]:
#train process

helper = Helper(dataDir=dataDir, continueWorking=continueWorking)
if continueWorking:
    loadedEpoch, constNoise, constLabels = helper.loadModel(gNetwork, dNetwork, genOptimizer, discrOptimizer, epoch)
else:
    loadedEpoch = 0
    constNoise = noise(numTestSamples)
    constLabels = getLabelsForGenerator(numTestSamples).long()

loadedEpoch_plus7 = loadedEpoch+7
# loadedEpoch_plus1 = loadedEpoch+1


def processDiscriminator(dataBatch, labelsFromBatch, labelsForGenerator):
    realSamples = Variable(imagesToDiscriminatorInput(dataBatch))
    if torch.cuda.is_available(): 
        realSamples = realSamples.cuda()
    generatedSamples = gNetwork(noise(realSamples.size(0)), labelsForGenerator).detach()
    # train
    loss, predictionReal, predictionGen = discriminatorTrain(discrOptimizer, realSamples, generatedSamples, labelsFromBatch, labelsForGenerator)
    return loss, predictionGen, predictionReal 


def processGenerator(dataBatch, labelsForGenerator):
    generatedSamples = gNetwork(noise(dataBatch.size(0)), labelsForGenerator)
    #train
    loss = generatorTrain(genOptimizer, generatedSamples, labelsForGenerator)
    return loss


def showAndLogImagesAndStats(epoch, numOfBatch, discriminatorLoss, generatorLoss, predictionGen, predictionReal):
    #images
    display.clear_output(True)
    labelsForGenerator = constLabels
    #getLabelsForGenerator(numTestSamples)
    testingImages = generatorOutputToImages(gNetwork(constNoise, labelsForGenerator.long())).data.cpu()
    helper.printLabels(labelsForGenerator)
    # helper.logOnBoardAndSaveImages(dataBatch, epoch, numOfBatch, numBatchesPerEpoch);
    helper.logOnBoardAndSaveImages(testingImages, epoch, numOfBatch, numBatchesPerEpoch);
    
    #stats 
    helper.showLossAndErrorsAndSaveInFile(
                    epoch, numEpochs, numOfBatch, numBatchesPerEpoch,
                    discriminatorLoss, generatorLoss, predictionGen, predictionReal 
                )


for epoch in range(loadedEpoch, min(loadedEpoch_plus7, numEpochs)):
    for numOfBatch, (dataBatch, labelsFromBatch) in enumerate(dataLoader):
        # for label in labelsFromBatch:
        #     if int(label) == 98:
        #         print(label)
        #         print('contain 98')
        
        # labelsForGenerator = labelsFromBatch
        labelsForGenerator = getLabelsForGenerator(len(labelsFromBatch))
        discriminatorLoss, predictionGen, predictionReal = processDiscriminator(dataBatch, labelsFromBatch, labelsForGenerator)

        # labelsForGenerator = labelsFromBatch
        labelsForGenerator = getLabelsForGenerator(len(labelsFromBatch))
        generatorLoss = processGenerator(dataBatch, labelsForGenerator)
        
        if numOfBatch % batchSize == 0:
            showAndLogImagesAndStats(epoch, numOfBatch, discriminatorLoss, generatorLoss, predictionGen, predictionReal)
        # save model
    helper.saveModel(gNetwork, dNetwork, genOptimizer, discrOptimizer, epoch+1, generatorLoss, discriminatorLoss, constNoise, constLabels) # save as epoch +1 because this is end of the epoch, and should be used as begin of the epoch+1

In [ ]:
#result images

numOfBatch=0
if epoch == numEpochs:
    for imageNum in range(numResultImagesPacks):
       someNoise = noise(numResultImagesInPack)
       testingImages = generatorOutputToImages(gNetwork(someNoise, getLabelsForGenerator(numResultImagesInPack))).data.cpu()
       helper.logOnBoardAndSaveImages(testingImages, epoch, numOfBatch, numBatchesPerEpoch, True, imageNum);
       """
       helper.showLossAndErrorsAndSaveInFile(
           epoch, numEpochs, numOfBatch, numBatchesPerEpoch,
           discriminatorLoss, generatorLoss, predictionGen, predictionReal 
       )
       """

In [ ]:
%tb


